# CSS Codes

## Background


### Stabilizer groups.

Let $G$ be a group and $X$ be a set. A group action $\varphi$ of $G$ on $X$ is a function $\varphi \colon G \times X \to X$ such that:
-  for all $x \in X$, we have $\varphi(\mathrm{id}_G, x) = x$;
-  for all $g,h \in G$, we have $\varphi(gh,x) = \varphi(g,\varphi(h,x))$.

Often we drop the notation $\varphi$ and simply write $gx$ to mean $\varphi(g,x)$.
The stabilizer of the group action of $G$ on $X$ is the set:
\begin{equation*}
    G_X = \{ g \in G | gx = x \}.
\end{equation*}
That is, the elements of $G$ that act like the identity on $X$.

**Example**
Consider the symmetric group $S_3$ whose elements are all the permutations of $3$ objects $0$, $1$ and $2$. For $\{a_0,a_1,a_2\} = \{0,1,2\}$, the permutation $0 \to a_0$, $1 \to a_1$, and $2 \to a_2$ can be represented by a tuple $(a_0,a_1,a_2)$. 

In [ ]:
class Permutation:
    """
    A permutation of {0, ..., n-1}, represented as a tuple A=(a0,a1, ..., an-1)
    where i -> A[i].
    """
    def __init__(self,
                 mapping: tuple[int]):
        self.n = len(mapping)
        if set(mapping) != set(range(self.n)):
            raise ValueError("A permutation must contain each number from 0 to n-1 exactly once.")
        self.mapping = mapping



There are four Pauli matrices: $I$, $X$, $Y$, and $Z$. These form a group under matrix multiplication where each matrix is of order 2. Thus, we may form tensor groups of Pauli matrices. 


One class of quantum error correction codes can be defined using tensor products of Pauli matrices. In particular, a _stabilizer code_ is the positive eigenspace of an abelian subgroup of tensors of Pauli matrices. There are three non-trivial Pauli matrices: $X$, $Y$, $Z$. An _$X$-type stabilizer_ is a tensor product that consists of $X$ matrices and identity matrices. Similarly, we define a _$Z$-type stabilizer_. A _CSS code_ is a stabilizer code with only $X$-type and $Z$-type generators. 

The 